# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#query_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
    #'address={0}&key={1}').format(target_city, gkey)
file = "../output_data/cities.csv"
cities_df = pd.read_csv(file)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/output?parameters
url = "https://maps.googleapis.com/maps/api/geocode/json?address="
query_url = url + "portland" + "&key=" + g_key
testerz = requests.get(query_url).json()
#print(json.dumps(testerz, indent=4))
#Store latitude and longitude in locations
locations = cities_df[["Latitude", "Longitude"]]
# Fill NaN values and convert to float
# rating = airport_df["Airport Rating"].astype(float)
# Plot Heatmap
fig = gmaps.figure()
# Create heat layer
humidity = cities_df['Humidity%']
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
# Add layer
fig.add_layer(heatmap_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
cities_df.head()

,City,Latitude,Longitude,Max Temp(F),Humidity%,Cloudiness%,Windspeed(MPH),Hemisphere
0,tanabe,33.73,135.37,86.00,70,40,4.70,North
1,longyearbyen,78.22,15.64,46.40,93,40,6.93,North
2,vaini,-21.20,-175.20,75.20,69,20,8.05,South
3,albany,42.60,-73.97,70.00,85,57,8.79,North
4,sao joao da barra,-21.64,-41.05,67.77,69,21,7.29,South


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [27]:
hotel_df = cities_df.loc[
    (cities_df["Max Temp(F)"]>70) &
    (cities_df["Max Temp(F)"]<80) &
    (cities_df["Windspeed(MPH)"]<10) &
    (cities_df["Cloudiness%"]==0)]
hotel_df = hotel_df.reindex(columns=hotel_df.columns.tolist() + ["Hotel Name"])
hotel_df

,City,Latitude,Longitude,Max Temp(F),Humidity%,Cloudiness%,Windspeed(MPH),Hemisphere,Hotel Name
91,udachnyy,66.42,112.40,71.60,37,0,2.24,North,NaN
148,moranbah,-22.00,148.05,76.12,33,0,4.38,South,NaN
149,rosario,-32.95,-60.64,71.60,56,0,8.05,South,NaN
168,kuryk,43.20,51.65,71.85,68,0,8.32,North,NaN
187,methoni,36.82,21.70,78.01,82,0,3.36,North,NaN
225,irtyshskiy,58.33,68.13,70.99,81,0,4.36,North,NaN
240,mozarlandia,-14.74,-50.57,70.09,39,0,7.09,South,NaN
246,sile,41.18,29.61,73.99,65,0,1.01,North,NaN
275,baruun-urt,46.68,113.28,70.81,52,0,4.07,North,NaN
276,komsomolskiy,40.43,71.72,75.20,38,0,4.70,North,NaN


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [25]:
locales = (hotel_df[['Latitude','Longitude']])

In [29]:
for index, row in locales.iterrows():
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location":f"{row[0]},{row[1]}",
        "radius":5000,
        "type":"lodging",
        "key":g_key
    }
    response = requests.get(url,params=params).json()
    results = response["results"]
    
    try:
        hotel_df.loc[index,"Hotel Name"] = results[0]["name"]
    except (KeyError,IndexError):
        pass
hotel_df = hotel_df.dropna()
hotel_df

,City,Latitude,Longitude,Max Temp(F),Humidity%,Cloudiness%,Windspeed(MPH),Hemisphere,Hotel Name
91,udachnyy,66.42,112.40,71.60,37,0,2.24,North,Gostinitsa Vilyuy
148,moranbah,-22.00,148.05,76.12,33,0,4.38,South,Direct Hotels - Monterey Moranbah
149,rosario,-32.95,-60.64,71.60,56,0,8.05,South,Holiday Inn Rosario
168,kuryk,43.20,51.65,71.85,68,0,8.32,North,Gostinitsa Kuryk - Otel' Dlya Vsekh Kategoriy ...
187,methoni,36.82,21.70,78.01,82,0,3.36,North,Methoni Beach Hotel
240,mozarlandia,-14.74,-50.57,70.09,39,0,7.09,South,Hotel Santa Mônica
246,sile,41.18,29.61,73.99,65,0,1.01,North,Seref Hotel
275,baruun-urt,46.68,113.28,70.81,52,0,4.07,North,Ахмадын амралт сувилал
276,komsomolskiy,40.43,71.72,75.20,38,0,4.70,North,Ikathouse
364,contamana,-7.33,-75.02,74.89,64,0,1.50,South,Shiringal Touristic Park


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [30]:
# Add marker layer ontop of heat map
markers=gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)


# Display figure
fig

NameError: name 'hotel_info' is not defined